# Vertex pipelines

**Learning Objectives:**

Use components from `google_cloud_pipeline_components` to create a Vertex Pipeline which will
  1. train a custom model on Vertex AI
  1. create an endpoint to host the model 
  1. upload the trained model and deploy to the endpoint for serving

## Overview

This notebook shows how to use the components defined in [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) in conjunction with an experimental `run_as_aiplatform_custom_job` method, to build a [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines) workflow that trains a [custom model](https://cloud.google.com/vertex-ai/docs/training/containers-overview), uploads the model, creates an endpoint, and deploys the model to the endpoint. 

We'll use the `kfp.v2.google.experimental.run_as_aiplatform_custom_job` method to train a custom model.

The google cloud pipeline components are [documented here](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.1.2/). From this [github page]() you can also find other examples in how to build a Vertex pipeline with AutoML [here](https://github.com/GoogleCloudPlatform/ai-platform-samples/tree/master/ai-platform-unified/notebooks/official/pipelines). You can see other available methods from the [Vertex AI SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html).

### Set up your local development environment and install necessary packages



In [131]:
!pip3 install --user google-cloud-aiplatform --upgrade
!pip3 install --user kfp google-cloud-pipeline-components --upgrade

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages. Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [73]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.6.4


#### Set your environment variables
Next, we'll set up our project variables, like GCP project ID, the bucket and region. Also, to avoid name collisions between resources created, we'll create a timestamp and append it onto the name of resources we create in this lab.

In [119]:
import os
from datetime import datetime

PROJECT = "munn-sandbox"
BUCKET = "munn-sandbox"
REGION = "us-central1"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

We'll save pipeline artifacts in a directory called `vertex_pipelines` within our bucket. Validate access to your Cloud Storage bucket by examining its contents. It should be empty at this stage. 

In [121]:
!gsutil ls -la gs://$BUCKET/vertex_pipelines

CommandException: One or more URLs matched no objects.


### Import libraries and define constants

In [122]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google import experimental
from kfp.v2.google.client import AIPlatformClient

## Define a pipeline that uses the components


Set some variables that will be used in constructing the args passed to the custom training job and setting pipeline params.

In [123]:
!gsutil ls gs://munn-sandbox/taxifare/data/taxi-train*

gs://munn-sandbox/taxifare/data/taxi-train-000000000000.csv


In [124]:
!gsutil ls gs://munn-sandbox/taxifare/data/taxi-valid*

gs://munn-sandbox/taxifare/data/taxi-valid-000000000000.csv


Next, you define a component with which the custom training job is run.  For this example, this component doesn't do anything (but run a print statement).

In [125]:
@component
def training_op(input1: str):
    print("training task: {}".format(input1))

Now, you define the pipeline.  

The `experimental.run_as_aiplatform_custom_job` method takes as args the component defined above, and the list of `worker_pool_specs`— in this case  one— with which the custom training job is configured. 
See [full function code here](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/google/experimental/custom_job.py)

Then, [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) components are used to define the rest of the pipeline: upload the model, create an endpoint, and deploy the model to the endpoint. (While not shown in this example, the model deploy will create an endpoint if one is not provided.)

In [126]:
# Output directory and job_name
OUTDIR=f"gs://{BUCKET}/taxifare/trained_model_{TIMESTAMP}"
JOB_NAME=f"taxifare_{TIMESTAMP}"

PYTHON_PACKAGE_URIS=f"gs://{BUCKET}/taxifare/taxifare_trainer-0.1.tar.gz"
MACHINE_TYPE="n1-standard-4"
REPLICA_COUNT=1
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-3:latest"
PYTHON_MODULE="trainer.task"

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=5000
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=f"gs://{BUCKET}/taxifare"
DATA_PATH=f"{GCS_PROJECT_PATH}/data"
TRAIN_DATA_PATH=f"{DATA_PATH}/taxi-train*"
EVAL_DATA_PATH=f"{DATA_PATH}/taxi-valid*"

In [127]:
@kfp.dsl.pipeline(name="taxifare--train-endpoint-deploy-" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = JOB_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest",
):
    train_task = training_op("taxifare model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        display_name=JOB_NAME,
        worker_pool_specs=[
            {
                "pythonPackageSpec": {
                    "executor_image_uri": PYTHON_PACKAGE_EXECUTOR_IMAGE_URI,
                    "package_uris": [PYTHON_PACKAGE_URIS],
                    "python_module": PYTHON_MODULE,
                    "args": [
                        f"--eval_data_path={EVAL_DATA_PATH}",
                        f"--output_dir={OUTDIR}",
                        f"--train_data_path={TRAIN_DATA_PATH}",
                        f"--batch_size={BATCH_SIZE}",
                        f"--num_examples_to_train_on={NUM_EXAMPLES_TO_TRAIN_ON}",
                        f"--num_evals={NUM_EVALS}",
                        f"--nbuckets={NBUCKETS}",
                        f"--lr={LR}",
                        f"--nnsize={NNSIZE}"
                    ],                    
                },
                "replica_count": f"{REPLICA_COUNT}",
                "machineSpec": {
                    "machineType": f"{MACHINE_TYPE}",
                },                
            }
        ],
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        artifact_uri=f"{OUTDIR}",
        serving_container_image_uri=serving_container_image_uri,
        serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    model_deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
        project=project,
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        machine_type="n1-standard-4",
    )

## Compile and run the pipeline

Now, you're ready to compile the pipeline:

In [128]:
from kfp.v2 import compiler  # noqa: F811

if not os.path.isdir("vertex_pipelines"):
    os.mkdir("vertex_pipelines")

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="./vertex_pipelines/train_upload_deploy.json"
)

The pipeline compilation generates the `train_upload_deploy.json` job spec file.

Next, instantiate an API client object:

In [129]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

Then, you run the defined pipeline like this: 

In [130]:
response = api_client.create_run_from_job_spec(
    "./vertex_pipelines/train_upload_deploy.json",
    pipeline_root=f"gs://{BUCKET}/vertex_pipelines",
    parameter_values={"project": PROJECT_ID},
)

Click on the generated link to see your run in the Cloud Console.  It should look something like this:

<a href="https://storage.googleapis.com/amy-jo/images/mp/train_endpoint_deploy.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/train_endpoint_deploy.png" width="75%"/></a>


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
- Delete Cloud Storage objects that were created.  Uncomment and run the command in the cell below **only if you are not using the `PIPELINE_ROOT` path for any other purpose**.
- Delete your deployed model: first, undeploy it from its *endpoint*, then delete the model and endpoint.


In [ ]:
# Warning: this command will delete ALL Cloud Storage objects under the PIPELINE_ROOT path.
# ! gsutil -m rm -r $PIPELINE_ROOT